***(Post-Training Quantization)***



Этот метод применяется к уже обученной модели без необходимости переобучения.

In [4]:
import torch
import torch.nn as nn
import torch.quantization

# Пример простой модели
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создаем модель
model = SimpleModel()

# Убедимся, что модель в режиме оценки
model.eval()

# Пример данных
input_fp32 = torch.randn(1, 784)

# Квантование модели
model.qconfig = torch.quantization.default_qconfig
torch.quantization.prepare(model, inplace=True)

# Калибровка модели (необходимо для посттренировочного квантования)
# Здесь можно использовать реальные данные для калибровки
with torch.no_grad():
    model(input_fp32)

# Применяем квантование
model_quantized = torch.quantization.convert(model, inplace=False)
print(model)
# Теперь модель квантована
print(model_quantized)

SimpleModel(
  (fc1): Linear(
    in_features=784, out_features=256, bias=True
    (activation_post_process): MinMaxObserver(min_val=-1.8477951288223267, max_val=1.6896419525146484)
  )
  (fc2): Linear(
    in_features=256, out_features=10, bias=True
    (activation_post_process): MinMaxObserver(min_val=-0.1601126790046692, max_val=0.3551177382469177)
  )
)
SimpleModel(
  (fc1): QuantizedLinear(in_features=784, out_features=256, scale=0.02785383351147175, zero_point=66, qscheme=torch.per_tensor_affine)
  (fc2): QuantizedLinear(in_features=256, out_features=10, scale=0.004056932404637337, zero_point=39, qscheme=torch.per_tensor_affine)
)


***(Quantization Aware Training)***


Этот метод предполагает обучение модели с учетом квантования, что может улучшить точность квантованной модели.

In [5]:
import torch
import torch.nn as nn
import torch.quantization

# Пример простой модели
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создаем модель
model = SimpleModel()

# Убедимся, что модель в режиме обучения
model.train()

# Добавляем квантование с осознанием обучения
model.qconfig = torch.quantization.default_qat_qconfig
torch.quantization.prepare_qat(model, inplace=True)

# Обучаем модель (здесь должен быть ваш код обучения)
# ...

# После обучения, конвертируем модель в квантованную
model.eval()
model_quantized = torch.quantization.convert(model, inplace=False)

# Теперь модель квантована
print(model_quantized)

SimpleModel(
  (fc1): QuantizedLinear(in_features=784, out_features=256, scale=1.0, zero_point=0, qscheme=torch.per_tensor_affine)
  (fc2): QuantizedLinear(in_features=256, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_tensor_affine)
)


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/utils.py:407: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


*** Квантование с использованием torch.quantization.quantize_dynamic***



Этот метод позволяет динамически квантовать модель, что особенно полезно для моделей с большим количеством линейных слоев.

In [6]:
import torch
import torch.nn as nn

# Пример простой модели
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Создаем модель
model = SimpleModel()

# Динамическое квантование
model_quantized = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)

# Теперь модель квантована
print(model_quantized)

SimpleModel(
  (fc1): DynamicQuantizedLinear(in_features=784, out_features=256, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc2): DynamicQuantizedLinear(in_features=256, out_features=10, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)
